### Sequential Model Based Optimization using the Tree Parzen Estimator for neural network model

There are four parts to an optimization problem:

1. Objective function: what we want to minimize
2. Domain space: values of the parameters over which to minimize the objective
3. Hyperparameter optimization function: constructs the surrogate function and chooses next values to evaluate
4. Trials: score, parameter pairs recorded each time we evaluate the objective function

In [1]:
# Good old pandas and numpy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


/home/fahimehb/miniconda3/envs/cplae_met/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import optuna

In [4]:
df = pd.read_csv('/home/fahimehb/Local/test/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [5]:

# Generate a simple data generator to use in the model
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [0 if label == 0 else 1 for label in df['HeartDisease']]
        self.features = df.drop(columns=['HeartDisease'], axis=1).values.tolist()

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_features(self, idx):
        return np.array(self.features[idx])

    def __getitem__(self, idx):
        batch_features = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_features, batch_y

In [6]:
train_data, val_data = train_test_split(df, test_size = 0.2, random_state = 42)

In [11]:
train, val = Dataset(train_data), Dataset(val_data)

In [21]:
# Build neural network model
def build_model(params):
    
    in_features = 20
    
    return nn.Sequential(
    
        nn.Linear(in_features, params['n_unit']),
        nn.LeakyReLU(),
        nn.Linear(params['n_unit'], 2),
        nn.LeakyReLU()
        
    )
 
 
# Train and evaluate the accuarcy of neural network model
def train_and_evaluate(param, model):
    
    df = pd.read_csv('/home/fahimehb/Local/test/heart.csv')
    df = pd.get_dummies(df)
    
    train_data, val_data = train_test_split(df, test_size = 0.2, random_state = 42)
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, "Adam")(model.parameters(), lr= 0.001)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(30):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)
            print(accuracy)

    return accuracy
  
 # Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy 
def objective(trial):

     params = {
            #   'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
            #   'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'n_unit': trial.suggest_int("n_unit", 4, 18)
              }
    
     model = build_model(params)
    
     accuracy = train_and_evaluate(params, model)

     return accuracy

In [22]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2023-01-03 15:13:16,626] A new study created in memory with name: no-name-bbc0cbbf-6998-4160-bc5b-824eae1225c1
[I 2023-01-03 15:13:49,667] Trial 0 finished with value: 0.5760869565217391 and parameters: {'n_unit': 5}. Best is trial 0 with value: 0.5760869565217391.
[I 2023-01-03 15:14:18,140] Trial 1 finished with value: 0.8532608695652174 and parameters: {'n_unit': 14}. Best is trial 1 with value: 0.8532608695652174.
[I 2023-01-03 15:14:50,889] Trial 2 finished with value: 0.8641304347826086 and parameters: {'n_unit': 5}. Best is trial 2 with value: 0.8641304347826086.
[I 2023-01-03 15:15:24,385] Trial 3 finished with value: 0.7717391304347826 and parameters: {'n_unit': 12}. Best is trial 2 with value: 0.8641304347826086.
[I 2023-01-03 15:15:58,230] Trial 4 finished with value: 0.7717391304347826 and parameters: {'n_unit': 11}. Best is trial 2 with value: 0.8641304347826086.
[I 2023-01-03 15:16:15,933] Trial 5 finished with value: 0.8532608695652174 and parameters: {'n_unit': 6}. B